In [24]:
from scipy import optimize
from scipy.integrate import solve_ivp

import numpy as np
import math


import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib import gridspec
import glob
import sys
import h5py
import cmasher as cmr

from scipy.interpolate import interp1d
import random



import matplotlib.patheffects as pe
import matplotlib.ticker as mticker
import matplotlib.patheffects as path_effects

matplotlib.rc('font', family='sans-serif', size=12)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['ytick.right'] = True
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['ytick.minor.visible'] = True
matplotlib.rcParams['lines.dash_capstyle'] = 'round'
matplotlib.rcParams['lines.solid_capstyle'] = 'round'
matplotlib.rcParams['figure.dpi'] = 200
cycle=plt.rcParams['axes.prop_cycle'].by_key()['color']


import re
from matplotlib import rc



In [25]:
# define parameters in the model, fiducial choices are K=1.0 and omega_phi = 200

K = 1.0

omega_phi = 200

# delta_t is the size of the timestep.
# delta_t and total number of timesteps must be chosen such that the steady-state PDF converges with respect to both parameters.
# fiducial values are listed below.

delta_t_in_Myrs = 1e-6

time_steps = 6000

# choose a ratio of cold to (mobile) warm phase fluid elements (or stochastic particles)
# this ratio effectively controls the Damkohler number for the model. Smaller ratio implies larger Da
cold_to_warm_ratio = 2.2 # this value corresponds to Da=430
# one can use the scaling relationship in Chen et al. 2025 to find the ratio needed for other Da choices

# limit the warm phase mobility
percentage_fixed_at_hot_phase = 0.95

# use the parameters above to calculate the number of stochastic particles initially in the cold and warm phases
number_of_cold_phase_fluid_particles = 300
number_of_mobile_hot_phase_fluid_particles = int(number_of_cold_phase_fluid_particles / cold_to_warm_ratio)
number_of_total_hot_phase_fluid_particles = int(number_of_mobile_hot_phase_fluid_particles / (1 - percentage_fixed_at_hot_phase))

# note that the total number of particles needs to be sufficiently large for the stead-state PDF to converge
# usually >3000 particles is sufficient. In this example we have just over 3000 particles.
total_number_of_particles = number_of_cold_phase_fluid_particles + number_of_total_hot_phase_fluid_particles

# the initial condition consists of two delta function of stochastic particles at the cold and warm phases
# note that in the Langevin model, phi values run from -1 to 1. This symmetry wirh respect to the origin is crucial
# later on we will map these phi values to the range of log(entropy) values appropriate for the cold and warm phase temperature
initial_phi_array = [1.0]*int(number_of_total_hot_phase_fluid_particles) + [-1.0]*int(number_of_cold_phase_fluid_particles)
initial_phi_array = np.array(initial_phi_array)
    
current_phi_array = initial_phi_array

# store values of <phi^2>, this is not crucial
average_phi_squared = np.zeros(time_steps)

# define a pressure.This affects the cold and warm phase temperatures and the strength of cooling
# Here we are choosing p/kb = 3000, appropriate for ISM condition
p_over_kb = 3000


# cooling and heating from Koyama & Inutsuka 2002, obtained by performing a piece-wise power law fit over ~50 logarithmically spaced temperature bins

T_range = [10.0, 16.244796890677783, 26.389342601937475, 42.868951064698464, 69.63974029624322, 113.12834366320207, 183.7746965387512, 298.538261891796, 
          484.96934285282003, 787.8228472849536, 1279.802213997954, 2079.0127026636515, 3377.313908791009, 5486.377848437104, 8912.509381337459, 10000.0, 12600.0,
          15890.0, 20020.0, 10000000.0, 100000000.0, 10000000000.0]

cooling_function_range = [8.946410010334062e-32, 3.916992415925812e-30, 4.4036281501965286e-29, 2.143881462872083e-28, 6.235148176998034e-28, 1.3205656560221116e-27, 2.300845357000786e-27, 
                         3.554858192054126e-27, 5.100682990533778e-27, 6.992886567893338e-27, 9.322013337019484e-27, 1.2214292098270173e-26, 1.5835184645629023e-26, 2.2758352514499377e-26, 
                         1.3251293935029884e-24, 1.59e-23, 6.16e-23, 1.86e-22, 1.55e-22, 2.92e-23, 2.61e-23, 2.61e-22]

heating_function_range = [2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 2e-26, 1.2597631645250691e-26,
                         7.921039905802994e-27, 4.9900149800249705e-27, 2e-32, 2.0000000000000003e-34, 2e-38]

T_range = np.array(T_range)
cooling_function_range = np.array(cooling_function_range)
heating_function_range = np.array(heating_function_range)

n_range = p_over_kb / T_range

kb = 1.380648e-16
mh = 1.6605e-24
unit_temp = 1e6
unit_rho = 1.0250266500000001e-27


Lambda_as_a_func_of_T = interp1d(T_range, cooling_function_range, fill_value='extrapolate')
Gamma_as_a_func_of_T = interp1d(T_range, heating_function_range, fill_value='extrapolate')

def net_cooling(T, p_over_kb):
    n = p_over_kb / T
    
    return n**2*Lambda_as_a_func_of_T(T) - n*Gamma_as_a_func_of_T(T)

# calculate the cold and warm phase equilibrium temperatures, either by root finding or by directly assigning

CNM_temp = optimize.root_scalar(lambda T: net_cooling(T, 3000), bracket=[10, 100]).root
WNM_temp = optimize.root_scalar(lambda T: net_cooling(T, 3000), bracket=[3e3, 1e4]).root

unstable_eq_temp = optimize.root_scalar(lambda T: net_cooling(T, 3000), bracket=[1e2, 3e3]).root

CNM_temp = 80.0
WNM_temp = 7000.0


# define the "warm phase" to be down to 0.8 times the warm phase equilibrium temperature
# stochastic particles above this temperature is in the warm phase and is subjected to limited mobility
# the choice of this cutoff temperature for the warm phase does not affect the stead-state PDF 
# as long as it is above 0.7 times the warm phase equilibrium temperature
cutoff_temperature = 0.8*WNM_temp


Lambda = interp1d(T_range, cooling_function_range, fill_value='extrapolate')

Gamma_heat_over_n = interp1d(T_range, heating_function_range/n_range, fill_value='extrapolate')

def dE_dt(T_in_K, p_over_kb):
    number_density = p_over_kb / T_in_K
    
    return number_density*(Lambda(T_in_K) - Gamma_heat_over_n(T_in_K))

dE_dt = np.vectorize(dE_dt)

# this function implements the net cooling function
def net_cooling(initial_T_in_K, p_over_kb, delta_t_in_seconds):
    dEnergy_dt = dE_dt(initial_T_in_K, p_over_kb)
    
    n = p_over_kb / initial_T_in_K
    gamma = 5.0/3.0
    kb = 1.380648e-16
    
    dT_dt = dEnergy_dt / ((1/(gamma - 1))*kb*n)
    
    new_T_in_K = initial_T_in_K - dT_dt*delta_t_in_seconds
    
    return new_T_in_K

net_cooling = np.vectorize(net_cooling)



In [26]:
# this function includes everything that happens in each timestep of the model evolution
def find_new_list_of_phi(K, omega_phi, delta_t_in_Myrs, old_list_of_phi, p_over_kb, total_number_of_particles):
    old_list_of_phi = np.array(old_list_of_phi)
    
    phi_B_list = np.zeros(len(old_list_of_phi))
    
    for i in range(len(phi_B_list)):
        if (old_list_of_phi[i] <= 0):
            phi_B_list[i] = -1.0
        else:
            phi_B_list[i] = 1.0
        
    average_phi_squared = np.average(old_list_of_phi**2)
    
    random_normal_range = np.zeros_like(old_list_of_phi)
    for j in range(len(random_normal_range)):
        random_normal_range[j] = np.random.normal()
        
    # phi runs from -1 to 1, but we need to calculate the range of entropy in physical units
    gamma = 5.0/3.0 # adiabatic index
    kb = 1.380648e-16 # boltzmann constant
    m_p = 1.67e-24 # mass of proton in grams
    X    = 0.7
    Z    = 0.02
    mu   = 1/(2*X + 0.75*(1-X-Z) + Z/2) # mean atomic weight
    
    # map log(entropy) values onto the fiducial range of values for phi (which is between -1 and 1)
    entropy_low_bound = (p_over_kb*kb) / ((p_over_kb / CNM_temp)*mu*m_p)**(5.0/3.0)
    entropy_high_bound = (p_over_kb*kb) / ((p_over_kb / WNM_temp)*mu*m_p)**(5.0/3.0)
    
    log_entropy_low_bound = np.log10(entropy_low_bound)
    log_entropy_high_bound = np.log10(entropy_high_bound)
    
    temporary_list_of_log_entropy = ((old_list_of_phi - (-1)) / (1 - (-1))) * (log_entropy_high_bound - log_entropy_low_bound) + log_entropy_low_bound
    temporary_list_of_entropy = 10**(temporary_list_of_log_entropy)
    temporary_list_of_T = p_over_kb / ((((p_over_kb*kb) / temporary_list_of_entropy)**(3.0 / 5.0)) / (mu*m_p))
    
    
    
    hot_phase_mask = temporary_list_of_T >= cutoff_temperature
    not_hot_phase_mask = temporary_list_of_T < cutoff_temperature
    
    hot_phase_index = np.where(hot_phase_mask)[0]
    number_of_hot_phase_to_evolve = np.floor(len(hot_phase_index)*(1-percentage_fixed_at_hot_phase))
    
    selected_indices = np.random.choice(hot_phase_index, size=int(number_of_hot_phase_to_evolve), replace=False)
    
    # evolve phi values according to the modified Langevin model (without the net cooling term yet)
    new_list_of_phi = old_list_of_phi
    new_list_of_phi[not_hot_phase_mask] = new_list_of_phi[not_hot_phase_mask] - 0.5*(1 + K*(1 - (average_phi_squared / phi_B_list[not_hot_phase_mask]**2)))*omega_phi*delta_t_in_Myrs*new_list_of_phi[not_hot_phase_mask] + np.sqrt(K*(1 - new_list_of_phi[not_hot_phase_mask]**2 / phi_B_list[not_hot_phase_mask]**2)*omega_phi*delta_t_in_Myrs*average_phi_squared)*random_normal_range[not_hot_phase_mask]
    new_list_of_phi[selected_indices] = new_list_of_phi[selected_indices] - 0.5*(1 + K*(1 - (average_phi_squared / phi_B_list[selected_indices]**2)))*omega_phi*delta_t_in_Myrs*new_list_of_phi[selected_indices] + np.sqrt(K*(1 - new_list_of_phi[selected_indices]**2 / phi_B_list[selected_indices]**2)*omega_phi*delta_t_in_Myrs*average_phi_squared)*random_normal_range[selected_indices]

    # implement reflecting boundaries
    greater_than_1_mask = new_list_of_phi >= 1
    new_list_of_phi[greater_than_1_mask] = 1
    less_than_minus1_mask = new_list_of_phi <= -1
    new_list_of_phi[less_than_minus1_mask] = -1
    
    
    # phi runs from -1 to 1, but we need to calculate the range of entropy in physical units
    gamma = 5.0/3.0 # adiabatic index
    kb = 1.380648e-16 # boltzmann constant
    m_p = 1.67e-24 # mass of proton in grams
    X    = 0.7
    Z    = 0.02
    mu   = 1/(2*X + 0.75*(1-X-Z) + Z/2) # mean atomic weight
    
    entropy_low_bound = (p_over_kb*kb) / ((p_over_kb / CNM_temp)*mu*m_p)**(5.0/3.0)
    entropy_high_bound = (p_over_kb*kb) / ((p_over_kb / WNM_temp)*mu*m_p)**(5.0/3.0)
    
    log_entropy_low_bound = np.log10(entropy_low_bound)
    log_entropy_high_bound = np.log10(entropy_high_bound)
    
    temporary_list_of_log_entropy = ((new_list_of_phi - (-1)) / (1 - (-1))) * (log_entropy_high_bound - log_entropy_low_bound) + log_entropy_low_bound
    temporary_list_of_entropy = 10**(temporary_list_of_log_entropy)
    temporary_list_of_T = p_over_kb / ((((p_over_kb*kb) / temporary_list_of_entropy)**(3.0 / 5.0)) / (mu*m_p))
    
    
    delta_t_in_seconds = delta_t_in_Myrs*3.154e13
    
    # implement net cooling
    new_list_of_T = net_cooling(temporary_list_of_T, p_over_kb, delta_t_in_seconds)
    
    
    new_list_of_entropy = (p_over_kb*kb) / ((p_over_kb / new_list_of_T)*mu*m_p)**(5.0/3.0)
    
    new_list_of_log_entropy = np.log10(new_list_of_entropy)
    
    new_list_of_phi = ((new_list_of_log_entropy - log_entropy_low_bound) / (log_entropy_high_bound - log_entropy_low_bound)) * (1 - (-1)) + (-1)
    
    # enforce the reflecting boundary again
    greater_than_1_mask = new_list_of_phi >= 1
    new_list_of_phi[greater_than_1_mask] = 1
    less_than_minus1_mask = new_list_of_phi <= -1
    new_list_of_phi[less_than_minus1_mask] = -1
    
            
    temporary_list_of_log_entropy = ((new_list_of_phi - (-1)) / (1 - (-1))) * (log_entropy_high_bound - log_entropy_low_bound) + log_entropy_low_bound
    temporary_list_of_entropy = 10**(temporary_list_of_log_entropy)
    temporary_list_of_T = p_over_kb / ((((p_over_kb*kb) / temporary_list_of_entropy)**(3.0 / 5.0)) / (mu*m_p))
    
    # output the array of temperatures so that we can plot the temperature PDF
    return new_list_of_phi, temporary_list_of_T
    


In [ ]:
# Apply the model for however many timesteps we want
for i in range(time_steps):
    current_phi_array, current_T_array = find_new_list_of_phi(K, omega_phi, delta_t_in_Myrs, current_phi_array, p_over_kb, total_number_of_particles)

    average_phi_squared[i] = np.average(current_phi_array**2)
    
    # plot the PDF once every 30 timesteps
    if (i % 30 == 0):
    
        plt.figure()
        plt.xlabel('log T')
        plt.ylabel(r'Count', fontsize=8)

        lavender = cmr.take_cmap_colors('cmr.lavender', 3, cmap_range=(0.0, 1.0), return_fmt='hex')

        n, bins, patches = plt.hist(np.log10(current_T_array),
                 bins=np.linspace(np.log10(CNM_temp), np.log10(WNM_temp), 50), 
                #density=True, stacked=True,
                alpha=0.5, histtype='stepfilled', color=lavender[2])
                
        plt.yscale('log')


        if (i//30 <= 9):
            image_name = 'modified_Langevin_model_PDF_00'+str(i//30) + '.png'
        elif (i//30 <= 99):
            image_name = 'modified_Langevin_model_PDF_0'+str(i//30) + '.png'
        else:
            image_name = 'modified_Langevin_model_PDF_'+str(i//30) + '.png'

        #plt.savefig(image_name, dpi=200, bbox_inches='tight', transparent=True, pad_inches=0)
        plt.show()
        

# save the stead-state temperature PDF as an npz file
np.savez('modified_Langevin_model_PDF.npz', stabilized_T_array=current_T_array, average_phi_squared=average_phi_squared)

